In [1]:
import os
import gc
import time

import keras.metrics
import numpy as np
import pandas as pd
import tensorflow as tf
import keras as k
from matplotlib import pyplot as plt
from tqdm import tqdm
import cv2
import albumentations
from utils import *
import skimage
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
# tf.compat.v1.disable_eager_execution()
print("Eager Execution Enabled:", tf.executing_eagerly())

2025-07-10 08:53:43.334808: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-07-10 08:53:43.474283: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-10 08:53:47.860154: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2025-07-10 08:53:47.860530: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

Num GPUs Available:  1
Eager Execution Enabled: True


2025-07-10 08:53:58.883405: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-07-10 08:53:59.180777: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-07-10 08:53:59.181040: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [2]:
model_input_shape = (224, 224)
model_feature_map_shape = tuple(np.asarray(model_input_shape) // 32)
no_classes = 20
alpha_loss = 0.75  # Class, SAL
beta_loss = 1.0 - alpha_loss # SAL
model_name = "model_softmax_part_class_alpha_0.75_mult"
full_dataset = False
schedule = False
schedule_change = 0.01
pool_layer = "softmax"  # "softmax" "class_softmax" "GeM" "mean" "max"
load_model = False
train = True
combined_loss_li = []
cce_loss_li = []
sal_loss_li = []
train_size = 0.9
use_max = False
learning_rate = (0.00005 * 0.5) if full_dataset else 0.00005 * 0.5
beta_lr_factor = 3.0 if full_dataset else 320 # Default 10
beta_start_val = 0.0
max_beta_val = 2.5  # default 2.5
epochs = 20 if full_dataset else 75
loss = "BCE" # "BCE" "CCE"
# base_model = tf.keras.applications.MobileNetV2(include_top=False, weights="imagenet")
# base_model = tf.keras.applications.ResNet50V2(include_top=False, weights="imagenet")
predictions_hist_train = []
predictions_hist_test = []
no_images = len(os.listdir("../../pascal/output/boxes")) if full_dataset else len(os.listdir("../../dataset/seg"))

no_models = 5

In [3]:
transform = albumentations.Compose([
    # albumentations.RandomCrop(width=224, height=224),
    albumentations.RandomCrop(width=model_input_shape[0], height=model_input_shape[1]),
    albumentations.HorizontalFlip(p=0.5),
    # albumentations.RandomBrightnessContrast(p=0.2, brightness_limit=0.1, contrast_limit=0.1),
])

In [4]:
classes_df = pd.read_csv("../../dataset/train_set.csv")


In [5]:
classes_mat = classes_df.drop(columns=["Id"]).to_numpy()


In [6]:
def load_image_and_mask(im_no):
    im_no = int(im_no)
    img = np.load(f"../../dataset/img/train_{im_no}.npy")
    seg = np.load(f"../../dataset/seg/train_{im_no}.npy")
    classes = classes_mat[im_no]
    return tf.convert_to_tensor(img, dtype=tf.uint8), tf.convert_to_tensor(seg, dtype=tf.uint8), tf.convert_to_tensor(classes, dtype=tf.uint8)

In [7]:
def tf_load_image_and_mask(im_no_t):
    image, mask, classes = tf.py_function(
        func=load_image_and_mask,    # Python function to call
        inp=[im_no_t],                 # Input tensor
        Tout=[tf.uint8, tf.uint8, tf.uint8]  # Output data types
    )

    # Set static shapes (important for TensorFlow pipelines)
    image.set_shape([model_input_shape[0], model_input_shape[1], 3])
    mask.set_shape([model_input_shape[0], model_input_shape[1], 1])
    return image, mask, classes

In [8]:
augment_image_and_mask = make_augement_image_and_mask(transform, model_input_shape)


In [9]:
def tf_augment_image_and_mask(image, mask, classes):
    image, mask = tf.py_function(
        func=augment_image_and_mask,    # Python function to call
        inp=[image, mask],                 # Input tensor
        Tout=[tf.float32, tf.uint8]  # Output data types
    )
    # Set static shapes (important for TensorFlow pipelines)
    image.set_shape([model_input_shape[0], model_input_shape[1], 3])
    mask.set_shape([model_input_shape[0], model_input_shape[1], no_classes + 1])
    return image, mask, classes

In [10]:
dataset = tf.data.Dataset.from_tensor_slices(np.arange(749))  # the number of dataset items I've got


2025-07-10 08:53:59.992943: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-07-10 08:53:59.995565: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-07-10 08:53:59.996775: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-07-10 08:53:59.997769: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

In [11]:
dataset = dataset.map(lambda img_id: tf_load_image_and_mask(img_id), num_parallel_calls=tf.data.AUTOTUNE)
dataset = dataset.map(lambda image, mask, classes: tf_augment_image_and_mask(image, mask, classes), num_parallel_calls=tf.data.AUTOTUNE)

In [12]:
def spatial_consistency_loss(score_map, heatmap, transformation):
    """
    Computes the Spatial Consistency Loss (SCL).
    Args:
        score_map (tf.Tensor): Current spatial score map (F^t_n), shape [batch_size, G, G, num_classes].
        heatmap (tf.Tensor): Running average heatmap (H^{t-1}_n), shape [batch_size, W, W, num_classes].
        transformation (tf.Tensor): Augmentation transformation applied to the input (e.g., crop, flip).
    Returns:
        tf.Tensor: The SCL value.
    """
    # Apply the transformation (e.g., cropping, flipping) to the heatmap
    transformed_heatmap = tf.image.resize(heatmap, score_map.shape[1:3], method='bilinear')
    # Compute the L1 distance
    scl = tf.reduce_mean(tf.abs(score_map - transformed_heatmap))
    return scl


@tf.function
def resize_to_input_tf(seg_masks):
    # Resize masks
    resized_masks = tf.image.resize(
        seg_masks,
        size=model_input_shape[:2],
        method=tf.image.ResizeMethod.BILINEAR
    )
    return resized_masks


resize_to_output_tf = make_resize_to_output_tf(model_input_shape, model_feature_map_shape)

def custom_loss_function(alpha=1.0, beta=1.0):
    """
    Combines Spatial Consistency Loss and Categorical Cross-Entropy Loss.
    Args:
        alpha (float): Weight for the CCE loss.
        beta (float): Weight for the SCL loss.
    Returns:
        function: A callable loss function to use during model training.
    """
    # mae = tf.keras.metrics.MeanSquaredError()

    def loss_function(y_true, y_pred, seg_masks, heatmap, test=False):
        """
        Args:
            y_true (tf.Tensor): Ground-truth class labels, shape [batch_size, num_classes].
            y_pred (tf.Tensor): Predicted class probabilities, shape [batch_size, num_classes].
            num_classes: Number of classes
            seg_masks: segmentation masks Shape: (batch, x, y, k)
            heatmap: Attention heatmaps
        Returns:
            tf.Tensor: The combined loss value.
        """
        # Compute categorical cross-entropy loss
        # print("Categorical", y_true.shape, y_pred.shape)
        if loss == "CCE":
            cce_loss = tf.keras.losses.CategoricalCrossentropy()(y_true, y_pred[:,:])
        elif loss == "BCE":
            cce_loss = tf.keras.losses.BinaryCrossentropy()(y_true, y_pred[:,:])
        # heatmap_resized = resize_to_input_tf(heatmap)
        # print(seg_masks[:,:,:,1:].shape)
        seg_masks_resized = resize_to_output_tf(seg_masks, heatmap.shape[1:])
        # print(heatmap.dtype, seg_masks_resized.dtype)
        # print(heatmap.shape, seg_masks_resized.shape)
        # print(seg_masks.numpy())
        # tf.print(heatmap_resized)
        # plt.imshow(to_ordinal(heatmap_resized.numpy()))
        # plt.show()
        # mae_loss = tf.keras.losses.MeanSquaredError()(seg_masks, heatmap_resized)
        # mae_loss = tf.keras.losses.CategoricalCrossentropy()(seg_masks, heatm
        if loss == "CCE":
            mae_loss = tf.keras.losses.CategoricalCrossentropy()(seg_masks_resized[:,:,:,:], heatmap[:,:,:,:])
        elif loss == "BCE":
            mae_loss = tf.keras.losses.BinaryCrossentropy()(seg_masks_resized[:,:,:,:], heatmap[:,:,:,:])
        # print(tf.keras.losses.CategoricalCrossentropy()(seg_masks_resized.numpy(), np.full(seg_masks_resized.shape, 1, dtype=np.float32)))
        # print(tf.keras.losses.CategoricalCrossentropy()(seg_masks_resized.numpy(), np.zeros(seg_masks_resized.shape) + 0.0001))
        # print(seg_masks_resized[5,:,:,15])

        # print(mae_loss, tf.keras.metrics.MeanSquaredError()(seg_masks, heatmap_resized))
        # m_iou.update_state(seg_masks, heatmap_resized)

        # # Compute spatial consistency loss
        # scl_loss = spatial_consistency_loss(score_map, heatmap, transformation)
        # if test:
        #     print(cce_loss, y_true.shape, y_pred.shape)
        #     # tf.print(y_pred[:,1:])
        #     tf.print("pred class:", tf.reduce_min(y_pred[:,1:]), tf.reduce_max(y_pred[:,1:]), tf.norm(y_pred[:,1:]))
        #     tf.print("pred seg:", tf.reduce_min(heatmap_resized), tf.reduce_max(heatmap_resized), tf.norm(heatmap_resized))
        # Combine the two losses
        total_loss = alpha * cce_loss + beta * mae_loss
        if test:
            return total_loss, cce_loss, mae_loss
        return total_loss

    return loss_function

In [13]:

def train_step(model, optimizer, images, seg_maps, labels, loss_fn, beta_optimizer):
    with tf.GradientTape(persistent=True) as tape:
        # Forward pass: predict class probabilities and score maps
        score_map, class_preds = model(images, training=True)

        # Compute the combined loss
        loss = loss_fn(labels, class_preds[:,1:], seg_maps, score_map, test=False)

    # Separate trainable variables
    beta_variables = []
    other_variables = []
    for var in model.trainable_variables:
        if "softmax_pooling" in var.name and "beta" in var.name:
            beta_variables.append(var)
        other_variables.append(var)

    # Backward pass: compute gradients for beta variables
    beta_gradients = tape.gradient(loss, beta_variables)

    # Backward pass: compute gradients for other variables
    other_gradients = tape.gradient(loss, other_variables)

    del tape

    # Apply gradients to beta variables with a higher learning rate
    if beta_gradients:
        beta_optimizer = tf.keras.optimizers.Adam(learning_rate=optimizer.learning_rate * beta_lr_factor)
        beta_optimizer.apply_gradients(zip(beta_gradients, beta_variables))
        del beta_optimizer

    # Apply gradients to other variables with the original learning rate
    optimizer.apply_gradients(zip(other_gradients, other_variables))

    return loss


def val_step(model, images, seg_maps, labels, loss_fn):
    score_map, class_preds = model(images, training=False)
    predictions_hist_test[-1] += list(np.dstack((class_preds.numpy()[:,1:], np.asarray(labels))))
    loss = loss_fn(labels, class_preds[:,1:], seg_maps[:,:,:,1:], score_map[:,:,:,1:], test=True)
    return loss


def train_model(model, dataset, epochs, optimizer, alpha, beta, losses=None, train_split=0.8, beta_optimizer=None):
    """
    Train the model using the custom train_step function.

    Args:
        model (tf.keras.Model): The neural network model.
        dataset (tf.data.Dataset): Dataset containing input images and labels.
        epochs (int): Number of training epochs.
        optimizer (tf.keras.optimizers.Optimizer): Optimizer for training.
        alpha (float): Weight for categorical cross-entropy loss.
        beta (float): Weight for spatial consistency loss.
    """
    if losses is None:
        losses = []
    best_loss = np.inf

    train_data = dataset.take(int(train_split * no_images)).batch(16).prefetch(tf.data.AUTOTUNE)
    test_data = dataset.skip(int(train_split * no_images)).batch(16).prefetch(tf.data.AUTOTUNE)
    # Initialize heatmaps as a dictionary for each image in the datase

    # model.compile(optimizer='adam')
    if pool_layer == "softmax":
        print(f"Softmax weight = {model.trainable_variables[-1]}")
    for epoch in range(epochs):
        if schedule:
            alpha -= schedule_change
            beta += schedule_change
        alpha = np.clip(alpha, 0.0, 1.0)
        beta = np.clip(beta, 0.0, 1.0)
        loss_fn = custom_loss_function(alpha, beta)
        predictions_hist_test.append([])
        print(f"\nEpoch {epoch + 1}/{epochs} \nAlpha = {alpha}, Beta = {beta}")
        epoch_loss = 0

        for step, (images, seg_maps, labels) in enumerate(tqdm(train_data)):
            # print(tf.shape(images))
            # batch_size = tf.shape(images)[0]

            # Perform a training step
            loss = train_step(
                model=model,
                optimizer=optimizer,
                images=images,
                labels=labels,
                seg_maps=seg_maps,
                loss_fn=loss_fn,
                beta_optimizer=beta_optimizer
            )
            epoch_loss += loss.numpy()

        avg_epoch_loss = epoch_loss / (step + 1)
        print(f"Epoch {epoch + 1}: Loss = {avg_epoch_loss}")
        losses.append(avg_epoch_loss)
        if "softmax" in pool_layer:
            print(f"Softmax weight = {model.trainable_variables[-1]}")


        epoch_test_loss, epoch_cce, epoch_sal = 0, 0, 0
        for step, (images, seg_maps, labels) in enumerate(test_data):

            combined_loss, cce_loss, sal_loss = val_step(model, images, seg_maps, labels, loss_fn)
            epoch_test_loss += combined_loss.numpy()
            epoch_cce += cce_loss.numpy()
            epoch_sal += sal_loss.numpy()

        avg_epoch_test_loss = epoch_test_loss / (step + 1)
        avg_epoch_cce = epoch_cce / (step + 1)
        avg_epoch_sal = epoch_sal / (step + 1)

        combined_loss_li.append(avg_epoch_test_loss)
        cce_loss_li.append(avg_epoch_cce)
        sal_loss_li.append(avg_epoch_sal)

        print(f"Epoch {epoch + 1}: Val loss = {avg_epoch_test_loss}, CCE loss = {avg_epoch_cce}, SAL loss = {avg_epoch_sal}")

        if avg_epoch_test_loss < best_loss:
            best_loss = avg_epoch_test_loss
            model.save_weights(f"weights/{model_name}.hdf5")
            # model.save("model_double_loss.keras", overwrite=True)
        # if epoch % 5 == 0:
        #     [(get_example(f"{no}.npy", epoch) if full_dataset else get_example(no, epoch)) for no in (range(17119, 17124) if full_dataset else range(743, 748))]

    # [(get_example(f"{no}.npy", epoch) if full_dataset else get_example(no, epoch)) for no in (range(17119, 17124) if full_dataset else range(743, 748))]
    return losses

In [ ]:
losses_all = []
cce_loss_all = []
sal_loss_all = []
predictions_hist_test_all = []
alpha_og, beta_og = alpha_loss, beta_loss
for mo_no in range(no_models):
    print(f"Model number: {mo_no} / {no_models}")
    alpha_loss, beta_loss = alpha_og, beta_og

    combined_loss_li = []
    cce_loss_li = []
    sal_loss_li = []
    predictions_hist_train = []
    predictions_hist_test = []
    if 'base_model' in locals():
        del base_model
    if 'model' in locals():
        del model
    if 'optimizer' in locals():
        del optimizer
    # if 'beta_optimizer' in locals():
    #     del beta_optimizer

    base_model = tf.keras.applications.ResNet50V2(include_top=False, weights="imagenet")
    model = create_model_with_spatial_scores(base_model, no_classes + 1, use_max, pool_layer, start_value=beta_start_val, beta_cap_value=max_beta_val)
    losses_hist = []

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    # beta_optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate * beta_lr_factor)

    losses_hist = train_model(model, dataset, epochs,
                optimizer,
                alpha_loss, beta_loss, losses=losses_hist, train_split=train_size)

    losses_all.append(losses_hist)
    sal_loss_all.append(sal_loss_li)
    cce_loss_all.append(cce_loss_li)
    predictions_hist_test_all.append(predictions_hist_test)

    if 'base_model' in locals():
        del base_model
    if 'model' in locals():
        del model
    if 'optimizer' in locals():
        del optimizer
    # if 'beta_optimizer' in locals():
    #     del beta_optimizer
    tf.keras.backend.clear_session()
    gc.collect()

Model number: 0 / 5
Softmax weight = <tf.Variable 'softmax_pooling/beta:0' shape=() dtype=float32, numpy=0.0>

Epoch 1/75 
Alpha = 0.75, Beta = 0.25


  0%|          | 0/43 [00:00<?, ?it/s]2025-07-10 08:54:03.701506: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8907
2025-07-10 08:54:05.056385: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.14GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2025-07-10 08:54:05.056411: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.14GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2025-07-10 08:54:05.171158: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.14GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean

Epoch 1: Loss = 0.684582395609035
Softmax weight = <tf.Variable 'softmax_pooling/beta:0' shape=() dtype=float32, numpy=-0.34499484>
Epoch 1: Val loss = 0.48820085525512696, CCE loss = 0.4515601634979248, SAL loss = 0.5981229305267334

Epoch 2/75 
Alpha = 0.75, Beta = 0.25


100%|██████████| 43/43 [00:45<00:00,  1.05s/it]


Epoch 2: Loss = 0.4663731344910555
Softmax weight = <tf.Variable 'softmax_pooling/beta:0' shape=() dtype=float32, numpy=-0.68974274>
Epoch 2: Val loss = 0.37547715902328493, CCE loss = 0.35070822238922117, SAL loss = 0.4497839629650116

Epoch 3/75 
Alpha = 0.75, Beta = 0.25


...

100%|██████████| 43/43 [01:20<00:00,  1.86s/it]


Epoch 7: Loss = 0.22861512906329576
Softmax weight = <tf.Variable 'softmax_pooling/beta:0' shape=() dtype=float32, numpy=-2.4119122>
Epoch 7: Val loss = 0.2530597150325775, CCE loss = 0.24172333180904387, SAL loss = 0.2870688736438751

Epoch 8/75 
Alpha = 0.75, Beta = 0.25


100%|██████████| 43/43 [01:20<00:00,  1.87s/it]


Epoch 8: Loss = 0.2086933012618575
Softmax weight = <tf.Variable 'softmax_pooling/beta:0' shape=() dtype=float32, numpy=-2.7556987>
Epoch 8: Val loss = 0.23352643549442292, CCE loss = 0.2264213055372238, SAL loss = 0.2548418253660202

Epoch 9/75 
Alpha = 0.75, Beta = 0.25


 21%|██        | 9/43 [00:17<01:04,  1.90s/it]

In [ ]:
np.shape(predictions_hist_test_all[-1])

In [ ]:
np.save(f"losses/multipleModels/{model_name}_cce_loss.npy", cce_loss_all)
np.save(f"losses/multipleModels/{model_name}_sal_loss.npy", sal_loss_all)
np.save(f"losses/multipleModels/{model_name}_predictions.npy", predictions_hist_test_all)

In [ ]:
model_name